<a href="https://colab.research.google.com/github/michellbrito/CSCE_614_Term_Project/blob/main/CSCE_614_CNN_for_MNIST_TPU_Acceleration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Package to install

In [ ]:
#!pip install --upgrade wandb
!wandb login 6508adb8701b189506a5541448dfc6f1e805edeb
#!pip install tensorflow

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


#Libraries

In [ ]:
#basics
from math import floor, sqrt, pi
from random import sample
import time
import timeit
#helpful progress bar
from tqdm.notebook import tqdm, trange

import warnings
warnings.filterwarnings("ignore")
from keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical

from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
import warnings
warnings.filterwarnings("ignore")  # Ignore some warning logs
from keras.models import Sequential

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

from sklearn.model_selection import cross_validate, train_test_split

import os

import pprint

import tensorflow_datasets as tfds



#Measuring Metrics
1) run the code below
2) click on the link
3) sign up if you need to
4) copy the code it gives you
5) paste it here
6) go back to the website, go to metrics. You will be able to see the metrics from there.

In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project="csce614")

epoch,1
loss,0.03704
accuracy,0.98857
_step,1
_runtime,781
_timestamp,1606608258


epoch,▁█
loss,█▁
accuracy,▁█
_step,▁█
_runtime,▁█
_timestamp,▁█


#TPU Initialization

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.93.80.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.93.80.218:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU')]


In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
with tf.device('/TPU:0'):
  c = tf.matmul(a, b)
print("c device: ", c.device)
print(c)

c device:  /job:worker/replica:0/task:0/device:TPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [ ]:
@tf.function
def matmul_fn(x, y):
  z = tf.matmul(x, y)
  return z

z = strategy.run(matmul_fn, args=(a, b))
print(z)

PerReplica:{
  0: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  1: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  2: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  3: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  4: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  5: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  6: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32),
  7: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
}


#Loading the MNIST Dataset

In [ ]:
def get_dataset(batch_size, is_training=True):
  split = 'train' if is_training else 'test'
  dataset, info = tfds.load(name='mnist', split=split, with_info=True,
                            as_supervised=True, try_gcs=True)

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0

    return image, label

  dataset = dataset.map(scale)

  # Only shuffle and repeat the dataset in training. The advantage to have a
  # infinite dataset for training is to avoid the potential last partial batch
  # in each epoch, so users don't need to think about scaling the gradients
  # based on the actual batch size.
  if is_training:
    dataset = dataset.shuffle(10000)
    dataset = dataset.repeat()

  dataset = dataset.batch(batch_size)

  return dataset

#CNN Neural Network with unoptimized hyperparameters

In [ ]:
#UNOPTIMIZED
def create_model2():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(50, 1, activation='relu', strides=(1,1), padding='same', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(75, 1, strides=(1,1), padding='same', activation='relu'),
       tf.keras.layers.MaxPool2D(pool_size=(2,2)), Dropout(0.25),
       tf.keras.layers.Conv2D(125, 1, strides=(1,1), padding='same', activation='relu'),
       tf.keras.layers.MaxPool2D(pool_size=(2,2)), Dropout(0.25),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(500, activation='relu'),
       tf.keras.layers.Dense(250, activation='relu'),
       tf.keras.layers.Dense(10)])

In [ ]:
#loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam'
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                # Anything between 2 and `steps_per_epoch` could help here.
                experimental_steps_per_execution = 50,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
batch_size = 128
steps_per_epoch = 60000 // batch_size
validation_steps = 10000 // batch_size

train_dataset = get_dataset(batch_size, is_training=True)
test_dataset = get_dataset(batch_size, is_training=False)

model.fit(train_dataset,
          epochs=15,
          steps_per_epoch=steps_per_epoch, callbacks=[WandbCallback()])

Epoch 1/15
468/468 [==============================] - 5s 10ms/step - loss: 0.1266 - accuracy: 0.9614
Epoch 2/15
468/468 [==============================] - 5s 10ms/step - loss: 0.0354 - accuracy: 0.9891
Epoch 3/15
468/468 [==============================] - 5s 10ms/step - loss: 0.0197 - accuracy: 0.9937
Epoch 4/15
468/468 [==============================] - 5s 10ms/step - loss: 0.0146 - accuracy: 0.9950
Epoch 5/15
468/468 [==============================] - 5s 10ms/step - loss: 0.0106 - accuracy: 0.9965
Epoch 6/15
468/468 [==============================] - 5s 10ms/step - loss: 0.0079 - accuracy: 0.9974
Epoch 7/15
468/468 [==============================] - 5s 10ms/step - loss: 0.0090 - accuracy: 0.9970
Epoch 8/15
468/468 [==============================] - 5s 10ms/step - loss: 0.0061 - accuracy: 0.9981
Epoch 9/15
468/468 [==============================] - 5s 10ms/step - loss: 0.0030 - accuracy: 0.9993
Epoch 10/15
468/468 [==============================] - 5s 10ms/step - loss: 0.0043 - accura

#HyperOpt: Optimizing hyperparameters - only run on GPU!

In [ ]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials


def optimize_cnn(hyperparameter):
  
  # Define model using hyperparameters 

  cnn_model = Sequential([Conv2D(50, kernel_size=hyperparameter['conv_kernel_size'], strides = hyperparameter['conv_stride'], padding='same', activation=hyperparameter['activation'], input_shape=(28,28,1)), 
          Conv2D(75, kernel_size=hyperparameter['conv_kernel_size'], strides = hyperparameter['conv_stride'], padding='same', activation=hyperparameter['activation']), 
          MaxPooling2D(pool_size=(2,2)), Dropout(hyperparameter['dropout_prob']),
          Conv2D(125, kernel_size=hyperparameter['conv_kernel_size'], strides = hyperparameter['conv_stride'], padding='same', activation=hyperparameter['activation']),
          MaxPooling2D(pool_size=(2,2)), Dropout(hyperparameter['dropout_prob']), 
          Flatten(),
          Dense(500, activation='relu'), Dropout(0.4),
          Dense(250, activation='relu'), Dropout(0.3),
          Dense(10, activation='softmax')])
  
  cnn_model.compile(optimizer=hyperparameter['optimizer'], loss='categorical_crossentropy', metrics=['accuracy'],)

  # create a training (50K samples) and validation (10K samples) subsets from training images.
  # Validation subset will be used to find the optimal hyperparameters
  train_X, train_y = train_images_3d[:50000], train_labels[:50000]
  valid_X, valid_y = train_images_3d[50000:], train_labels[50000:]

  _ = cnn_model.fit(train_X, to_categorical(train_y), epochs=2, batch_size=128, verbose=0)
  # Evaluate accuracy on validation data
  performance = cnn_model.evaluate(valid_X, to_categorical(valid_y), verbose=0)

  print("Hyperparameters: ", hyperparameter, "Accuracy: ", performance[1])
  print("----------------------------------------------------")
  # We want to minimize loss i.e. negative of accuracy
  return({"status": STATUS_OK, "loss": -1*performance[1], "model":cnn_model})
  

# Define search space for hyper-parameters
space = {
    # The kernel_size for convolutions:
    'conv_kernel_size': hp.choice('conv_kernel_size', [1, 3, 5]),
    # Uniform distribution in finding appropriate dropout values
    'dropout_prob': hp.uniform('dropout_prob', 0.1, 0.35),
    #choice of stride
    'conv_stride': hp.choice('conv_stride',[1,2]),
    #choice of activation
    'activation': hp.choice('activation', ['relu','selu','tanh']),
    # Choice of optimizer 
    'optimizer': hp.choice('optimizer', ['Adam', 'sgd']),
}

trials = Trials()

# Find the best hyperparameters
best = fmin(
        optimize_cnn,
        space,
        algo=tpe.suggest,
        trials=trials,
        max_evals=25,
    )

print("==================================")
print("Best Hyperparameters", best)

# You can retrain the final model with optimal hyperparameters on train+validation data

# Or you can use the model returned directly
# Find trial which has minimum loss value and use that model to perform evaluation on the test data
test_model = trials.results[np.argmin([r['loss'] for r in trials.results])]['model']

performance = test_model.evaluate(test_images_3d, to_categorical(test_labels))

print("==================================")
print("Test Accuracy: ", performance[1])

#CNN Neural Network with optimized hyperparametrs

In [ ]:
#OPTIMIZED
def create_model2():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(50, 3, activation='relu', strides=(1,1), padding='same', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(75, 3, strides=(1,1), padding='same', activation='relu'),
       tf.keras.layers.MaxPool2D(pool_size=(2,2)), Dropout(0.2538798802417247),
       tf.keras.layers.Conv2D(125, 3, strides=(1,1), padding='same', activation='relu'),
       tf.keras.layers.MaxPool2D(pool_size=(2,2)), Dropout(0.2538798802417247),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(500, activation='relu'),
       tf.keras.layers.Dense(250, activation='relu'),
       tf.keras.layers.Dense(10)])

In [ ]:
with strategy.scope():
  model = create_model2()
  model.compile(optimizer='adam',
                # Anything between 2 and `steps_per_epoch` could help here.
                experimental_steps_per_execution = 50,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
batch_size = 128
steps_per_epoch = 60000 // batch_size
validation_steps = 10000 // batch_size

train_dataset = get_dataset(batch_size, is_training=True)
test_dataset = get_dataset(batch_size, is_training=False)

model.fit(train_dataset,
          epochs=15,
          steps_per_epoch=steps_per_epoch, callbacks=[WandbCallback()])

Epoch 1/15
468/468 [==============================] - 4s 8ms/step - loss: 0.1489 - accuracy: 0.9522
Epoch 2/15
468/468 [==============================] - 4s 8ms/step - loss: 0.0447 - accuracy: 0.9856
Epoch 3/15
468/468 [==============================] - 4s 8ms/step - loss: 0.0311 - accuracy: 0.9900
Epoch 4/15
468/468 [==============================] - 4s 8ms/step - loss: 0.0244 - accuracy: 0.9925
Epoch 5/15
468/468 [==============================] - 4s 8ms/step - loss: 0.0210 - accuracy: 0.9931
Epoch 6/15
468/468 [==============================] - 4s 8ms/step - loss: 0.0179 - accuracy: 0.9944
Epoch 7/15
468/468 [==============================] - 4s 8ms/step - loss: 0.0145 - accuracy: 0.9953
Epoch 8/15
468/468 [==============================] - 4s 8ms/step - loss: 0.0143 - accuracy: 0.9956
Epoch 9/15
468/468 [==============================] - 4s 8ms/step - loss: 0.0131 - accuracy: 0.9959
Epoch 10/15
468/468 [==============================] - 4s 8ms/step - loss: 0.0120 - accuracy: 0.9963

#Wandb FINISH

In [ ]:
wandb.run.finish()

epoch,14
loss,0.00712
accuracy,0.99766
_step,29
_runtime,188
_timestamp,1606608472


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,▇▃▂▂▁▁▁▁▁▁▁▁▁▁▁█▃▂▂▂▂▂▂▂▁▁▁▁▁▁
accuracy,▂▆▇▇███████████▁▆▇▇▇▇▇▇▇██████
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇███
